Export all FEFLOW Kx values as Center in an ASCII table and read the table into a dataframe with pandas

In [ ]:
import pandas as pd
import numpy as np

#Import dask for parallelization of merge
import dask
import dask.dataframe as dd

feflow = pd.read_fwf('Carrot_Kx.dat', colspecs = 'infer', infer_nrows = 12)
feflow.head(5)

Append new indexing column equal to the sum of X,Y,Z truncated to 0 decimal places

In [ ]:
import math as math

#Truncate values in feflow dataframe
#Change feflow XYZ type to int
type_dict = {'X': int, 'Y': int, 'Z': int}
feflow = feflow.astype(type_dict)
feflow.head(5)

Import RQD Block Model as CSV

In [ ]:
mfq = pd.read_csv('MFQ_RQD.csv')
mfq.head(5)

Define mathematical transformation to convert RQD values to hydraulic conductvity

In [ ]:
def transform(x):
    return x/1000 

Create New Dataframe (mfqkx) and apply transformation of RQD values to Kx on MFQ dataframe

In [ ]:
import numpy as np
mfq_transform = mfq.apply(lambda x: transform(x) if x.name == 'RQD%' else x)
mfqkx = mfq_transform[['X','Y','Z']].copy()
mfqkx['X'] = mfqkx['X'].astype(int)
mfqkx['Y'] = mfqkx['Y'].astype(int)
mfqkx['Z'] = mfqkx['Z'].astype(int)
mfqkx['RQD'] = pd.Series(mfq_transform['RQD%'])
print(mfqkx.head(5))


Use dask to merge datasets based on common XYZ location

In [ ]:
#Convert dataframes from pandas to dask
# - NOTE: Dask should not be used prior to this point as it handles local processes (i.e. applying custom function) very slowly
d_feflow = dd.from_pandas(feflow, npartitions=1)
d_mfqkx = dd.from_pandas(mfqkx, npartitions=1)

merge = dd.merge(d_feflow, d_mfqkx,how='left',on=['X','Y','Z'])
print(merge.head(5))

Export final dataframe to .xlsx (Excel) file that can be read for parameter linkage in FEFLOW

In [ ]:
#Remove rows where RQD is NaN
df = merge.dropna()
print(df.head())
df.to_csv('merge.csv')